# Академия Аналитиков Авито

# ML для аналитиков

# Занятие №16

- На предыдущих занятиях вы 

- Научились тестировать на наличие автокорреляции

- Узнали что такое стационарность

- Познакомились с `SARIMAX` моделью


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
from tqdm import tqdm

import scipy as sp

import statsmodels.formula.api as smf
import statsmodels.api as sm

from statsmodels.graphics.tsaplots import plot_acf 
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.stats.diagnostic import acorr_ljungbox

import scipy as sp
import seaborn as sns

# Модельки
from statsmodels.tsa.arima.model import ARIMA
# Тесты
from statsmodels.tsa.stattools import adfuller, kpss, acf

import prophet as fp 
import datetime
import time

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import graph_objs as go

from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics
from prophet.plot import add_changepoints_to_plot

from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor


# инициализируем plotly
init_notebook_mode(connected = True)

pd.set_option('display.max_columns', None) # Чтобы показывались все колонки
%matplotlib inline

In [ ]:
def adf_test(timeseries):
    print ('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print (dfoutput, '\n Null Hypothesis: The series has a unit root.')
    
def kpss_test(timeseries):
    print ('Results of KPSS Test:')
    kpsstest = kpss(timeseries, regression='c', nlags="auto")
    kpss_output = pd.Series(kpsstest[0:3], index=['Test Statistic','p-value','Lags Used'])
    for key,value in kpsstest[3].items():
        kpss_output['Critical Value (%s)'%key] = value
    print (kpss_output, '\n Null Hypothesis: The process is trend stationary.')
    
# опишем функцию, которая будет визуализировать все колонки dataframe в виде line plot
def plotly_df(df, title = ''):
    data = []

    for column in df.columns:
        trace = go.Scatter(
            x = df.index,
            y = df[column],
            mode = 'lines',
            name = column
        )
        data.append(trace)

    layout = dict(title = title, template='plotly_white')
    fig = dict(data = data, layout = layout)
    iplot(fig, show_link=False)
    
# Функция для получения данных 'mdape', 'mape', 'mtape' для модели по методу имитированных исторических прогнозов
def perf_metrics_d(fp_model):
    fp_df_cv = cross_validation(fp_model, initial='730.25 days', period='180 days', horizon = '180 days', parallel="processes")
    res = performance_metrics(fp_df_cv,rolling_window = 1)
    return res    

## План
- Prophet (базовая модель)
    - Формат подаваемых данных 
    - Базовая модель и параметры по умолчанию 
    - Декомпозиция временного ряда на составляющие
- Ошибка прогноза и кросс-валидация
    - Типы ошибок
    - Кросс-валидация
- Prophet (гипертюнинг параметров)
    - Точки изменения тренда
    - Сезонные компоненты
    - Эффекты праздников
    - Добавление экзогенной переменной
    - Подбор параметров
- Random Forest
    - Feature Engineering
    - Подбор параметров
    - Фит

## Prophet (базовая модель)
### Формат подаваемых данных

[**Документация**](https://facebook.github.io/prophet/docs/quick_start.html#python-api)


[**Данные с прошлого занятия: продажи магазинов Kaggle**](https://www.kaggle.com/competitions/playground-series-s3e19/data)

Данные о продажах в различных *вымышленных* обучающих модулей из *вымышленных* магазинов под брендом Kaggle в разных (*настоящих*) странах. Этот набор данных полностью синтетический, но содержит множество эффектов, которые вы видите в реальных данных, например, эффект выходных и праздников, сезонность, ковид и так далее. 

In [ ]:
# Получение и обработка исходных данных.
sales_train_raw = pd.read_csv('train_data.csv', parse_dates=['date'])

In [ ]:
sales_train = pd.pivot_table(sales_train_raw, index='date', values='sales', aggfunc=sum).reset_index()
sales_train.columns = ['ds', 'y']

In [ ]:
validation_horizon = 180
data = sales_train[sales_train['ds']<=(max(sales_train['ds']) - datetime.timedelta(days=validation_horizon))]
data_evaluation = sales_train

In [ ]:
# Посмотрим на первоначальные данные
plotly_df(data.set_index('ds')[['y']])

На вход принимается dataframe с двумя колонками:
- ds — время, поле должно быть типа date или datetime,
- y — числовой показатель, который мы хотим предсказывать.

Это условные обозначения, принятые в prophet. Использование каких–либо других имен приведет к ошибке при вызове соответствующих функций. Мы уже заранее подготовили нужный нам кусочек данных с правльным названием колонок. 

In [ ]:
data.head()

### Базовая модель и параметры по умолчанию

Модель прогнозирования Prophet - это модель прогнозирования временных рядов, предназначенная для работы с общими для бизнеса *особенностями* временных рядов (множественные сезонности, изменения тренда, праздники). 

<img src= "https://facebookexperimental.github.io/Robyn/img/prophet_decomp.png" width="50%" >


В основе этой методологии лежит процедура подгонки аддитивных регрессионных моделей со следующими четырьмя основными компонентами: 

**$$y(t) = g(t) + s(t) + h(t) + \varepsilon_t$$**

- **Тренд $g(t)$** — это кусочно-линейная или кусочная логистическая кривая роста. С линейной кривой все понятно. Логистическая же функция вида $g(t) = \frac{C}{1+\exp(-k(t-b))}$ позволяет моделировать рост с насыщением, когда при увеличении показателя снижается темп его роста.


- **Сезонные компоненты $s(t)$** отвечают за моделирование периодических изменений, связанных с недельной и годовой сезонностью. Моделируются [рядами Фурье](https://ru.wikipedia.org/wiki/%D0%A0%D1%8F%D0%B4_%D0%A4%D1%83%D1%80%D1%8C%D0%B5).


- **Компонента $h(t)$** отвечает за заданные пользователем **праздники и аномальные периоды**, то есть периоды непредсказуемого поведения метрики. "Праздниками" в FBProphet являются в том числе и нерегулярные периоды, дни акций, период COVID19 тоже можно отразить в данной компоненте. Праздники представлены в виде dummy variables.


- **Ошибка $\textbf{$\varepsilon_t$}$** - нормально распределенные случайные процессы. Компонента содержит информацию, которая не учтена моделью.

- Рассмотрим четыре шага для прогноза с помощью базовой модели

In [ ]:
# 1. Создадим объект класса Prophet (все параметры модели задаются в конструкторе класса, используем дефолтные)
m = fp.Prophet()

# 2. Проведем обучение
m.fit(data)

# 3. Cоздадим таблицу с датами, охватывающими даты истории + "горизонт" для прогнозирования
future = m.make_future_dataframe(periods=validation_horizon)

# 4. Получим предсказания на датах, полученных в предыдщуем шаге
forecast = m.predict(future)

In [ ]:
# forecast - это таблица pd.DataFrame, в которой хранятся значения рассчитанных на основе модели m величин
# Включены тренд, сезонные компоненты модели, предсказанные значения, а также верхние и нижние границы доверительных интервалов

# Вот так выглядят первые несколько предсказанных значений метрики и их (принятые по умолчанию) 80%-ные доверительные границы:
forecast.head()

In [ ]:
# Встроенный метод для отображения прогноза
# Отражается с 80% интервалом неопределенности (зависящий от тренда и шума)
fig1 = m.plot(forecast)


### Декомпозиция временного ряда на составляющие




In [ ]:
# Можно отразить компоненты прогноза 
# По умолчанию вы увидите тренд, годовую сезонность и недельную сезонность временного ряд
# Если вы включите праздники, вы увидите их здесь

fig2 = m.plot_components(forecast)

- Давайте же оценим качество нашей базовой модели, посчитав ошибку MAPE. 
- Напомню, что в прошлый раз мы получили следующие результаты MAPE для моделей семейства SARIMAX:
    - Model ARIMA MAPE is ..
    - Model SARIMAX MAPE is ..
    - **Model SARIMA MAPE is ..**
    
    

In [ ]:
# преобразуем формат для джойна табличек
data_evaluation['ds'] = pd.to_datetime(data_evaluation['ds'])
data['ds'] = pd.to_datetime(data['ds'])
data_w_error = data_evaluation.merge(forecast[['ds','yhat']], on = ['ds'], how='left')

In [ ]:
data_w_error

In [ ]:
# считаем ошибку
data_w_error['mape'] = abs(data_w_error['y'] - data_w_error['yhat'])/data_w_error['y']


print('Prophet MAPE is: ', np.mean(data_w_error[data_w_error['ds']>max(data['ds'])]['mape']))

In [ ]:
data_w_error.columns=['ds','y', 'forecast_p','mape']
data_w_error.loc[data_w_error['ds']<=max(data['ds']),'forecast_p'] = np.nan
data_w_error

In [ ]:
data_predicted_class_15 = pd.read_csv('data_predicted_class_1.csv')

In [ ]:
data_sarima_prophet = data_predicted_class_15.merge(data_w_error[['forecast_p']], left_index=True, right_index=True)





In [ ]:
plotly_df(data_sarima_prophet[['y_validation','forecast_arima','forecast_x','forecast_s','forecast_p']])


- Видно, что Prophet справился с задачей сильно лучше, чем SARIMA
- Но какие еще виды ошибок есть? Как более честно сравнить две модели? 

## Ошибка прогноза и кросс-валидация

### Типы ошибок

- Ошибки, зависящие от масштаба: 
$$\text{ Mean absolute error: }  MAE = mean(|y_t - \hat{y_t}|)$$
$$\text{ Root mean squared error: }  RMSE = \sqrt{mean(y_t - \hat{y_t})^2}$$

- При этом, важно сказать, что RMSE стремится сократить **среднее** расстояние между точками, а MAE - **медианное** расстояние. Соответственно RMSE сильнее штрафует более сильные отклонения факта от прогноза, чем MAE. 

- Приведем приимер:



In [ ]:
def plot_time_series(time, values, label):
    plt.figure(figsize=(10,6))
    plt.plot(time, values)
    plt.xlabel("Time", fontsize=20)
    plt.ylabel("Value", fontsize=20)
    plt.title(label, fontsize=20)
    plt.grid(True)

In [ ]:
pos_seas = 0
neg_seas = 5
error_std = 50

def gen_ts(pos_seas,neg_seas,error_std, trend=True):
    # Just a random pattern
    time = np.arange(50)
    values_hat = np.where(time < 10, (time+pos_seas)**3, np.where(time>45, -(time+neg_seas)**2, time*2 ))
    # Repeat the pattern 5 times
    seasonal_hat = []
    for i in range(5):
        for j in range(50):
            seasonal_hat.append(values_hat[j])

    noise_hat = np.random.normal(loc = 0, scale = error_std, size = 250)
    seasonal_hat += noise_hat
    if trend==True:
        seasonal_trend_hat = seasonal_hat + np.arange(250)*5
    else: 
        seasonal_trend_hat = seasonal_hat
    
    return seasonal_trend_hat


df_gen = pd.concat([pd.Series(np.arange(250), name = 'ds'),
                       pd.Series(np.ravel(gen_ts(pos_seas,neg_seas,error_std)), name = 'y')], axis=1)
        
        

plotly_df(df_gen.set_index('ds'))        
        

In [ ]:
df_gen = pd.concat([pd.Series(np.arange(250), name = 'ds'),
                    pd.Series(np.ravel(gen_ts(0,9,50)), name = 'y'),
                    pd.Series(np.ravel(gen_ts(5,10,100)), name = 'yhat')], axis=1)    

plotly_df(df_gen.set_index('ds'))        
        

In [ ]:
df_gen['AE'] = np.abs(df_gen['yhat']-df_gen['y'])
df_gen['SE'] = (df_gen['yhat']-df_gen['y'])**2

In [ ]:
plotly_df(df_gen.set_index('ds'))

In [ ]:
print('MAE: ', np.mean(np.abs(df_gen['yhat']-df_gen['y'])))
print('RMSE: ', np.sqrt(np.mean((df_gen['yhat']-df_gen['y'])**2)))

In [ ]:
for pos_seas in [1,2,3,4,5]:
    
    df_gen = pd.concat([pd.Series(np.arange(250), name = 'ds'),
                    pd.Series(np.ravel(gen_ts(0,9,50)), name = 'y'),
                    pd.Series(np.ravel(gen_ts(pos_seas,12,100)), name = 'yhat')], axis=1) 

    mae = np.mean(np.abs(df_gen['yhat']-df_gen['y']))
    rmse = np.sqrt(np.mean((df_gen['yhat']-df_gen['y'])**2))
    
    print('for pos_seas = {}, MAE: {}, RMSE: {}, difference: {}'.format(pos_seas,mae,rmse, rmse-mae))

- Ошибки, не зависящие от масштаба: 

$$\text{ Mean absolute percentage error: }  MAPE = mean(\frac{|y_t - \hat{y_t}|}{y_t})$$

- Несмотря на свою привлекательность, у МАPE есть два явных минуса:

    - если значния $y_t$ колеблются около нуля, ошибка сильно завышается или же вообще не может быть посчитана.

    - MAPE обычно сильно выше для случаев, когда прогноз переоценивает метрику (Прогноз > Факт), так как в знаменателе MAPE стоит именно фактическая метрика. Из этого следует ограниченность MAPE снизу, те при условии прогноза выше нуля, максимальная ошибка "снизу" будет 100%. "Сверху" же MAPE никак не ограничивает метрику, соответственно ошибка может стремиться к бесконечности.
    
- Приведем пример: 
 



In [ ]:
df_gen = pd.concat([pd.Series(np.arange(250), name = 'ds'),
                    pd.Series(np.ravel(gen_ts(0,-45,50, trend=False)/800), name = 'y'),
                    pd.Series(np.ravel(gen_ts(1,-45,20, trend=False)/800), name = 'yhat')], axis=1)    

plotly_df(df_gen.set_index('ds'))        
        

In [ ]:
df_gen['APE'] = np.abs(df_gen['yhat']-df_gen['y'])/df_gen['y']
plotly_df(df_gen.set_index('ds'))  

In [ ]:
df_gen = pd.concat([pd.Series(np.arange(250), name = 'ds'),
                    pd.Series(np.ravel(gen_ts(0,-30,20, trend=True)+200), name = 'y'),
                    pd.Series(np.ravel(gen_ts(-1.5,-45,20, trend=True)+200), name = 'yhat')], axis=1)    

df_gen['y_yhat_diff'] = abs(df_gen['y']-df_gen['yhat'])

plotly_df(df_gen.set_index('ds'))        
        

In [ ]:
df_gen['APE'] = np.abs(df_gen['yhat']-df_gen['y'])/df_gen['y']
plotly_df(df_gen.set_index('ds'))  

- Чтобы исправить эту проблему, можно использовать следующие типы ошибок: 

$$\text{ Median absolute percentage error: }  MDAPE = median(\frac{|y_t - \hat{y_t}|}{y_t})$$

$$\text{ Mean absolute scaled error: }  MASE = mean(\frac{y_t - \hat{y_t}}{\frac{1}{T-1}\sum^T_{t=2}|y_t-y_{t-1}|})$$ 

$$\text{ Mean absolute scaled error (for seasonal data): }  MASE = mean(\frac{y_t - \hat{y_t}}{\frac{1}{T-m}\sum^T_{t=m+1}|y_t-y_{t-m}|})$$

 $$\text{ Symmetric mean absolute percentage error: }  SMAPE = mean(\frac{2*|y_t - \hat{y_t}|}{y_t+ \hat{y_t}})$$



- Проверим, как работают эти ошибки

- MDAPE

$$\text{ Median absolute percentage error: }  MDAPE = median(\frac{|y_t - \hat{y_t}|}{y_t})$$

In [ ]:
plotly_df(df_gen.set_index('ds'))  

In [ ]:
mape = np.mean(df_gen['APE'])
mdape = np.median(df_gen['APE'])

print('MAPE: {}, MDAPE: {}'.format(mape,mdape))

- MDAPE действительно помогло сократить ошибку до "нормального" уровня, однако стоит быть очень осторожным, ведь MDAPE не учитывает совсем периоды, в которые прогноз сильно отличается от факта. 
- Рекомендуется сравнивать MAPE и MDAPE скорее для случаев, когда хочется обнаружить такие периоды. 

- MASE

$$\text{ Mean absolute scaled error (for seasonal data): }  MASE = mean(\frac{|y_t - \hat{y_t}|}{\frac{1}{T-m}\sum^T_{t=m+1}|y_t-y_{t-m}|})$$

In [ ]:
mase = np.mean(np.abs(df_gen['y']-df_gen['yhat']))/np.mean(np.abs(df_gen['y']-df_gen['y'].shift(50)))
print('MASE: ', mase)

- Ключевое отличие MASE от MAPE заключается в том, что MASE сравнивает полученный прогноз с, так называемым, наивным прогнозом (naive forecast): 
    - Наивный прогноз - это простое повторение последнего наблюдения в качестве прогноза на будущие периоды: $y_{t} = \hat{y_{t-1}}$.
    - Для сезональных данных наивным прогнозом будет являться повторение последнего наблюдения из предыдущего сезона: $y_{t}= \hat{y_{t-m}}$
    
    
- Таким образом, знаменателем MASE является MAE для наивного прогноза, а не фактическое значение, как в MAPE. Соответственно, MASE не подвержен тем проблемам, которые есть у MAPE, однако и смысл у MASE другой. 


- MASE > 1 означает, что ошибка выбранного прогноза больше, чем ошибка наивного прогноза, те выбранный прогноз хуже, чем наивный. MASE < 1, соответственно, говорит об обратном.
- Сравнивая несколько моделей, модель с наименьшим MASE является наилучшей. 

- SMAPE 

 $$\text{ Symmetric mean absolute percentage error: }  SMAPE = mean(\frac{2*|y_t - \hat{y_t}|}{y_t+ \hat{y_t}})$$

In [ ]:
plotly_df(df_gen.set_index('ds'))  

In [ ]:
df_gen['smape'] = 2*np.abs(df_gen['yhat']-df_gen['y'])/(df_gen['y']+df_gen['yhat'])

In [ ]:
plotly_df(df_gen.set_index('ds'))  

In [ ]:
mape = np.mean(df_gen['APE'])
mdape = np.median(df_gen['APE'])
smape = np.mean(df_gen['smape'])

print('MAPE: {}, MDAPE: {}, SMAPE: {}'.format(mape,mdape,smape))

- Можно заметить, что за счет наличия факта и прогноза в числителе, SMAPE частично сглаживает аномальные негативные отклонения факта от прогноза, однако не по прежнему имеет тенденцию завышать ошибку для значений близких к нулю. 

- В рамках нашего курса мы все же чаще будем опираться на MAPE и MDAPE, так как для нас одинково критично завысить или занизить прогноз. Сравнивая MDAPE и MAPE мы будем пытаться привести их к схожим значениям. 

### Кросс-валидация

- Окей, мы изучили кучу разных типов ошибок, поняли, чем они отличаются и попробовали оценить качество модели методом Out Of Sample (OOS). Однако, тру саентисты скажут, что модель может сильно переобучиться в момент обучения, соответственно и OOS ошибка может быть сильно занижена.

- И тру саентисты будут правы) Как известно от переобучения есть одно простое средство: кросс-валидация, которая работает немного особенным способом в Time-series. 

- При работе с временными рядами применяют несколько модификаций классической кросс-валидации. Чаще всего различают **rolling-window cross-valiadation** (A) и **expanding window cross-validation** (B) . В пакете fbrophet, в частности, реализован метод expanding window (Simulated Historical Forecasts, SHF).

![image.png](https://www.researchgate.net/profile/Alireza-Shojaei/publication/326380749/figure/fig1/AS:647993657159681@1531505135948/Visual-represenation-of-cross-validation-methods-used-A-Evaluation-based-on-a-fixed.png)


- Expanding window может применяться чаще, когда ряд имеет ярко выраженный сезонный паттерн и стабильный тренд, так как в этом случае первые наблюдения ряда содержат потенциальную информацию о будущих значениях. 

- Rolling window полезно, когда у нас довольно волатильный ряд или когда наиболее недавняя история более важна для прогнозирования (высокая корреляция с наиболее недавними лагами).

Expanding window            |  Rolling window
:-------------------------:|:-------------------------:
![](https://robotwealth.com/wp-content/uploads/2020/05/sma_viz_expanding-1.gif)  |  ![](https://robotwealth.com/wp-content/uploads/2020/05/sma_viz_rolling2.gif)


Функция FBProphet <code>cross_validation()</code> принимает следующие параметры:

- <code>initial</code> ("начальный отрезок") - исторические наблюдения за initial дней , образуют обучающие данные для подгонки соответствующей модели (по умолчанию составляет 3×H)


- <code>horizon</code> ("горизонт прогноза") - следующие за initial H точек наблюдений, для которых строится прогноз


- <code>period</code> - период, на который увеличивается initial перед следующим прогнозом (по умолчанию составляет H/2)


<img src= "https://i.ibb.co/nsbjc6k/2023-04-04-16-04-49.png">



In [ ]:
df_cv = cross_validation(m, initial='730.5 days', period='180 days', horizon = '180 days', parallel="processes")

In [ ]:
# результатом кросс-валидации является таблица с тренировочными данными за исключением периода initial, 
# однако теперь для каждого наблюдения подсчитан прогноз
df_cv

In [ ]:
# функция performance_metrics подсчитывает основные виды ошибок. параметр rolling_window отвечает за долю наблюдений
# в фолде на основе которых подсчитывается ошибка. Мы используем все данные фолда для подсчета ошибки. 
res = performance_metrics(df_cv,rolling_window = 1)
res

In [ ]:
pb_metrics = perf_metrics_d(m)

print(f"Performance metrics of model:\nMDAPE = {round(pb_metrics.at[0,'mdape'],4)}\nMAPE = {round(pb_metrics.at[0,'mape'],4)}\n")


In [ ]:
# преобразуем формат для джойна табличек
data_evaluation['ds'] = pd.to_datetime(data_evaluation['ds'])
data['ds'] = pd.to_datetime(data['ds'])
data_w_error = data_evaluation.merge(forecast[['ds','yhat']], on = ['ds'], how='left')

In [ ]:
data_w_error['APE'] = abs(data_w_error['y']-data_w_error['yhat'])/data_w_error['y']

plotly_df(data_w_error.set_index('ds')[['y', 'yhat','APE']], title = 'Fact vs Forecast')


## Prophet (гипертюнинг параметров)

[Полный список параметров модели FBProphet c описанием.](https://facebook.github.io/prophet/docs/diagnostics.html)

Все параметры отвечают за каждый из компонентов FBProphet: 
- Trend
    - **changepoint_prior_scale**
    - **changepoint_range**    
    - growth 
    - changepoints
    - n_changepoints

- Seasonality
    - **seasonality_prior_scale**
    - **seasonality_mode**  
    - yearly_seasonality 
    - weekly_seasonality

- Holidays
    - **holidays**
    - **holidays_prior_scale** 
  
В документации Профета рекомендуется настраивать именно параметры, выделенные жирным. Разберем их подробнее:

- **changepoint_prior_scale** — параметр, задающий чувствительность автоматического механизма обнаружения точек излома в тренде временного ряда y (0.05 по умолчанию). Более высокие значение позволят иметь больше таких точек излома (что одновременно увеличит риск переобучения модели)

- **changepoint_range** — доля исторических данных (начиная с самого первого наблюдения), по которым будут оценены точки излома. По умолчанию составляет 0.8 (т.е. 80% наблюдений)

- **seasonality_prior_scale** — параметр, определяющий выраженность сезонных компонент модели (10 по умолчанию). Более высокие значения приведут к более “гибкой” модели, а низкие — к модели со слабее выраженными сезонными эффектами. Этот параметр можно задать отдельно для каждого типа сезонности с помощью функции add_seasonality()

- **seasonality_mode** — режим моделирования сезонных компонент. Принимает два возможных значения: 'additive' (аддитивный, задан по умолчанию) и 'multiplicative' (мультипликативный)

- **holidays** — таблица, содержащая два обязательных столбца: holiday (текстовая переменная с названиями "праздников") и ds (даты). По желанию в такую таблицу можно добавить еще два столбца — *lower_window* и *upper_window*, которые задают отрезок времени вокруг соответствующего события. Так, например, при "lower_window = -2" в модель будут добавлены 2 дня, предшествующие соответствующему событию

- **holidays_prior_scale** — параметр, определяющий выраженность эффектов 'праздников' и других важных событий (10 по умолчанию)

### Точки изменения тренда

[Почитать подробнее в доках](https://facebook.github.io/prophet/docs/trend_changepoints.html#adjusting-trend-flexibility)

In [ ]:
print('Default params:\n')
# Посмотрим, какое максимально возможное количество точек изменения тренда доступно в дефолтных параметрах 
print(f'n_changepoints = {m.n_changepoints}\n')

# Посмотрим, на каком периоде исторических данных профет будет выставлять точки
print(f'changepoint_range = {m.changepoint_range}\n')

# Если изменения тренда являются чрезмерными или недостаточными, вы можете отрегулировать силу влияния changepoint_prior_scale. 
# По умолчанию этот параметр установлен на 0,05. Увеличение его сделает общую тенденцию более гибкой, уменьшение менее гибкой
print(f'changepoint_prior_scale = {m.changepoint_prior_scale}\n')

In [ ]:
fig = m.plot(forecast)

# Изобразить эти автоматически обнаруженные точки излома можно с помощью функции *add_changepoints_to_plot()
# Добавим сетку с точками изменения тренда (профет берет не все 25, а лишь отбирает не больше 25)
a = add_changepoints_to_plot(fig.gca(), m, forecast)

- Что будет, если изменить **n_changepoints** (сделать показатель равным 4 или 100)?
- Что конкретно менятся в модели, если изменить **changepoint_prior_scale** (сделать показатель равным 0.005, 0.025, 0.5)? 
- Для чего нам может потребоваться сделать **changepoint_range** равным 0.95?

In [ ]:
m = fp.Prophet(
#     n_changepoints=100
#     changepoint_prior_scale=0.0001
#     changepoint_range=0.6
)
m.fit(data)

future = m.make_future_dataframe(periods=45)

forecast = m.predict(future)

fig = m.plot(forecast)
a = add_changepoints_to_plot(fig.gca(), m, forecast)

### Сезонные компоненты


[Почитать здесь](https://facebook.github.io/prophet/docs/seasonality,_holiday_effects,_and_regressors.html#fourier-order-for-seasonalities)

- Сезональные модели временных рядов делятся на два основных типа — аддитивные и мультипликативные.

**$$y(t) = g(t) + s(t) + h(t) + \varepsilon_t$$**
**$$y(t) = g(t) * s(t) * h(t) * \varepsilon_t$$**

- Первый из них применяется в случаях, когда амплитуда сезонных колебаний приблизительно постоянна. Если же эта амплитуда заметно изменяется во времени (обычно возрастает), то строят мультипликативную модель.


![image.png](https://www.researchgate.net/publication/348592737/figure/fig2/AS:981645804961802@1611054006728/Examples-for-multiplicative-and-additive-relationship-between-time-series-components.png)

[Про мультипликативную сезональность](https://facebook.github.io/prophet/docs/multiplicative_seasonality.html)



In [ ]:
plotly_df(data.set_index('ds')[['y']])


In [ ]:
print('Default params:\n')
print(f'seasonality_mode = {m.seasonality_mode}\n')

print(f'seasonality_prior_scale = {m.seasonality_prior_scale}\n')

In [ ]:


m_4 = fp.Prophet(
#     seasonality_mode = 'multiplicative'
#     seasonality_prior_scale = 0.1
)

m_4.fit(data)

future = m_4.make_future_dataframe(periods=365*2)

forecast = m_4.predict(future)
fig6 = m.plot(forecast)

In [ ]:
fig6_5 = m_4.plot_components(forecast)

### Эффекты праздников

- Для наших целей воспользуемся таблицей holidays_events среди [таблиц датасета](https://www.kaggle.com/competitions/favorita-grocery-sales-forecasting/data)  - c ее помощью создадим датафрейм с праздниками под формат fbprophet

In [ ]:
holiday_df = pd.read_csv('holidays_events.csv',parse_dates=['date'])
holiday_df = holiday_df.rename(columns={'date' : 'ds','description':'holiday'})

In [ ]:
holiday_df

In [ ]:
print('Default params:\n')
print(f'holidays_prior_scale = {m.holidays_prior_scale}\n')


In [ ]:
# Посмотрим, что будет при учете праздников

m_holidays = fp.Prophet(
    holidays = holiday_df
    ,holidays_prior_scale = 0.01
)

m_holidays.fit(data)

future = m_holidays.make_future_dataframe(periods=365*2)

forecast = m_holidays.predict(future)
fig = m_holidays.plot(forecast)

In [ ]:
fig = m_holidays.plot_components(forecast)

- Также важным параметром, влияющим на прогноз является <code>holidays_prior_scale</code>.  Обычно параметр изменяется в пределах [0.01, 10], при этом, чем выше этот параметр, тем меньше регуляризация, а значит сильнее выражены эффекты "праздника". Дефолтное значение параметра - 10 (отсутствие регуляризации).

### Добавление нового предиктора

- Дополнительные регрессоры могут быть добавлены к линейной части модели с использованием add_regressor метода. 

- Столбец со значением регрессора должен присутствовать как во фреймах данных, так и в прогнозе. 


- Попробуем добавить дополнительный регрессор из прошлого занятия - Финансовый Индекс MSCI



In [ ]:
df_oil = pd.read_csv('oil.csv', parse_dates=['date'])
df_oil = df_oil.rename(columns={"date": "ds",'dcoilwtico':'oil'}).dropna()



In [ ]:
df_oil

In [ ]:
exog_vars = ['oil']

data_x = data[['ds', 'y']].merge(df_oil[['ds']+exog_vars], 
                    on='ds', how='left')
data_evaluation_x = data_evaluation[['ds', 'y']].merge(df_oil[['ds']+exog_vars],
                                          on='ds', how='left')

data_x[exog_vars] = data_x[exog_vars].interpolate(method='linear', axis=0,limit_direction='both')
data_evaluation_x[exog_vars] = data_evaluation_x[exog_vars].interpolate(method='linear', axis=0,limit_direction='both')

data_x

In [ ]:
m = fp.Prophet()

m.add_regressor('oil', mode='multiplicative')

m.fit(data_x)

future = m.make_future_dataframe(periods=validation_horizon)
future = future.merge(data_evaluation_x[['ds','oil']],how='left', on = 'ds')

forecast = m.predict(future)
fig = m.plot(forecast)
a = add_changepoints_to_plot(fig.gca(), m, forecast)





In [ ]:
fig = m.plot_components(forecast)

In [ ]:
fig = m.plot_components(forecast)

### Подбор параметров

In [ ]:
param_grid = {  
    'changepoint_prior_scale': [0.01, 0.1, 0.5],
    'seasonality_prior_scale': [0.1, 10.0],
    'seasonality_mode':['additive', 'multiplicative'],
    'holidays_prior_scale':[0.1,10.0]
}

# Создадим все комбинации параметров
all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
mapes = []  # будем записывать ошибки в этот лист 

# кросс валидация
for params in tqdm(all_params):
    m = fp.Prophet(**params).fit(data)  # Fit model with given params
    df_cv = cross_validation(m, initial='730.25 days', period='180 days', horizon = '180 days', parallel="processes")
    df_p = performance_metrics(df_cv, rolling_window=1)
    mapes.append(df_p['mape'].values[0])

# Find the best parameters
tuning_results = pd.DataFrame(all_params)
tuning_results['mape'] = mapes
tuning_results.sort_values(by='mape')

In [ ]:
best_params = all_params[tuning_results.sort_values(by='mape').index[0]]
best_params

- попробуем сделать тюнинг параметров с учетом внешнего фактора

In [ ]:
param_grid = {  
    'changepoint_prior_scale': [0.01, 0.5],
    'seasonality_prior_scale': [0.1, 10.0],
    'seasonality_mode':['additive', 'multiplicative'],
    'holidays_prior_scale':[0.1,10.0]
}

# Создадим все комбинации параметров
all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
mapes = []  # будем записывать ошибки в этот лист 

# кросс валидация
for params in tqdm(all_params):
    m = fp.Prophet(**params, holidays=holiday_df)  # Fit model with given params
    m.add_regressor('oil', mode=params['seasonality_mode'])
    m.fit(data_x)
    df_cv = cross_validation(m, initial='730.25 days', period='180 days', horizon = '180 days', parallel="processes")
    df_p = performance_metrics(df_cv, rolling_window=1)
    mapes.append(df_p['mape'].values[0])

# Find the best parameters
tuning_results = pd.DataFrame(all_params)
tuning_results['mape'] = mapes
tuning_results.sort_values(by='mape')

- Видно, что с внешним фактором результат получается сильно лучше. Включим его в модель.

In [ ]:
# Посмотрим, что будет при учете праздников

m_holidays = fp.Prophet(
    holidays = holiday_df,
    **best_params
)

m_holidays.add_regressor('oil', mode='multiplicative')

m_holidays.fit(data_x)

future = m_holidays.make_future_dataframe(periods=validation_horizon)
future = future.merge(data_evaluation_x[['ds','oil']],how='left', on = 'ds')


forecast = m_holidays.predict(future)
fig = m_holidays.plot(forecast)
a = add_changepoints_to_plot(fig.gca(), m_holidays, forecast)


In [ ]:
# преобразуем формат для джойна табличек
data_evaluation['ds'] = pd.to_datetime(data_evaluation['ds'])
data['ds'] = pd.to_datetime(data['ds'])
data_w_error = data_evaluation.merge(forecast[['ds','yhat']], on = ['ds'], how='left')

In [ ]:
# считаем ошибку
data_w_error['mape'] = abs(data_w_error['y'] - data_w_error['yhat'])/data_w_error['y']
print('Prophet MAPE is: ', np.mean(data_w_error[data_w_error['ds']>max(data['ds'])]['mape']))

In [ ]:
data_w_error.columns=['ds','y', 'forecast_p','mape']
data_w_error.loc[data_w_error['ds']<=max(data['ds']),'forecast_p'] = np.nan

In [ ]:
data_predicted_class_15 = pd.read_csv('data_predicted_class_15.csv')
data_sarima_prophet = data_predicted_class_15.merge(data_w_error[['forecast_p']], left_index=True, right_index=True)


In [ ]:
plotly_df(data_sarima_prophet[['y_validation','forecast_s','forecast_p']])


- Давайте сравним модели SARIMAX и Prophet:
    - Model SARIMA MAPE is ..
    - **Model Prophet MAPE is ..**

## Random Forest
### Feature Engineering

In [ ]:
# Получение и обработка исходных данных.
sales_train_raw = pd.read_csv('train_data.csv', parse_dates=['date'])

sales_train = pd.pivot_table(sales_train_raw, index='date', values='sales', aggfunc=sum).reset_index()
sales_train.columns = ['ds', 'y']

validation_horizon = 180
data = sales_train[sales_train['ds']<=(max(sales_train['ds']) - datetime.timedelta(days=validation_horizon))]
data_evaluation = sales_train

In [ ]:
holiday_df['holiday_flag']=1

In [ ]:
import re

def add_datepart(dff, fldname, holidays, lags=7, drop=True, errors="raise"):
    df = dff.copy()
    fld=df[fldname]
    targ_pre = re.sub('ds', '', fldname)
    
    # date attributes
    attr = ['Month', 'Day', 'Dayofweek', 'Dayofyear',
            'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
    for n in attr: 
        df[targ_pre + n] = getattr(fld.dt, n.lower())
    df['Week']=(df['Dayofyear']/7).apply(np.ceil)
    
    df['Is_week_end'] = df['Dayofweek'].apply(lambda x: 1 if x in [5,6] else 0)
    
    # lags
    for lag in range(1,lags+1):
        df['y_lag_'+str(lag)] = df['y'].shift(lag)
    df = df.dropna()
    
    # diff
    df['y_diff_1'] = df['y'].diff()
    df['y_diff_2'] = df['y'].diff(2)
    df['y_diff_7'] = df['y'].diff(7)    
    df = df.dropna()
    
    # moving average
    df['y_ma_mean'] = df['y'].rolling(7).mean()
    df['y_ma_std'] = df['y'].rolling(7).std()
    df = df.dropna()    
    
    #holidays
    df = df.merge(holidays[['ds','holiday_flag']], on='ds', how='left')
    df['holiday_flag'] = df['holiday_flag'].fillna(0)
    
    if drop: df.drop(fldname, axis=1, inplace=True)

    return df
        
data = add_datepart(data_evaluation, 'ds',holiday_df)


In [ ]:
data

In [ ]:
def split_vals(a,n): 
    return a[:n].copy(), a[n:].copy()

n_trn = int(len(data)-180)
x = data.drop('y',axis = 1)
y = data[['y']]
X_train, X_test = split_vals(x, n_trn)
y_train, y_test = split_vals(y, n_trn)

X_train.shape, y_train.shape, X_test.shape

### Подбор параметров

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 20, stop = 800, num = 10)]
# Number of features to consider at every split
max_features = [1, 'sqrt',0.5]
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4,3,5]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

In [ ]:
# Use the random grid to search for best hyperparameters
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, np.ravel(y_train))


In [ ]:
rf_random.best_params_

### Фит модели

In [ ]:
best_random = rf_random.best_estimator_

foreacst_rf = best_random.predict(X_test)
mape_err = np.mean(abs(foreacst_rf-y_test['y'])/y_test['y'])

print(f'MAPE: {mape_err}')
   

In [ ]:
forecast_rf_df = pd.DataFrame(foreacst_rf)
forecast_rf_df.rename(columns = {0:'foreacst_rf'},inplace = True)
forecast_rf_df.index=range(len(data_evaluation)-180,len(data_evaluation))
forecast_rf_df.tail(10)

In [ ]:
data_evaluation

In [ ]:
data_predicted_class_15 = pd.read_csv('data_predicted_class_15.csv')

data_sarima_prophet_rf = data_predicted_class_15[['y_validation','forecast_s']].merge(
    data_w_error[['forecast_p']], left_index=True, right_index=True).merge(
    forecast_rf_df, how='left',left_index=True, right_index=True)


In [ ]:
# data_sarima_prophet_rf['foreacst_rf']=data_sarima_prophet_rf['foreacst_rf'].shift(3)


In [ ]:
plotly_df(data_sarima_prophet_rf)

- Давайте сравним модели SARIMAX и Prophet:
    - Model SARIMA MAPE is ...
    - Model Prophet MAPE is ...
    - **Model Random Forest is ...**